# Tarea 1

Implementar la clase `OisCashflow`.

## Configuración Inicial

In [1]:
from finrisk import QC_Financial_3 as Qcf
from dataclasses import dataclass
from enum import Enum
import pandas as pd

In [2]:
class BusCal(Enum):
    NY = 1
    SCL = 2

In [3]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    """
    """
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2019, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(11, 11, 2019))
        cal.add_holiday(Qcf.QCDate(28, 11, 2019))
        cal.add_holiday(Qcf.QCDate(25, 12, 2019))
        cal.add_holiday(Qcf.QCDate(20, 1, 2020))
        cal.add_holiday(Qcf.QCDate(17, 2, 2020))
        cal.add_holiday(Qcf.QCDate(10, 4, 2020))
        cal.add_holiday(Qcf.QCDate(25, 5, 2020))
        cal.add_holiday(Qcf.QCDate(3, 7, 2020))
        cal.add_holiday(Qcf.QCDate(7, 9, 2020))
        
        # cal.add_holiday(Qcf.QCDate(30, 9, 2020)) Este no corresponde y se detectó en los tests.
        
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    return cal

## `Qcf.time_series`

In [4]:
ts = Qcf.time_series()

fecha1 = Qcf.QCDate(13, 1, 1969)
ts[fecha1] = 19690113

fecha2 = Qcf.QCDate(14, 1, 1969)
ts[fecha2] = 19690114

print(f'ts[fecha1]: {ts[fecha1]}\n')

dl = Qcf.time_series_dates(ts)
for d in dl:
    print(d)
print()
    
vl = Qcf.time_series_values(ts)
for v in vl:
    print(v)

ts[fecha1]: 19690113.0

13-1-1969
14-1-1969

19690113.0
19690114.0


## Clase `OisCashflow`

La clase `OisCashflow` es una [`dataclass`](https://realpython.com/python-data-classes/). La tarea consiste en:

- Implementar los 3 métodos de la clase que están con `pass`.
- Implementar la función `present_value`.
- Implementar la función `set_expected_rate`.

In [5]:
@dataclass  # syntactic sugar
class OisCashflow:
    start_date: Qcf.QCDate
    end_date: Qcf.QCDate
    settlement_date: Qcf.QCDate
    notional: float
    currency: Qcf.QCCurrency
    amortization: float
    amort_is_cashflow: bool
    interest_rate: Qcf.QCInterestRate
    on_index: Qcf.InterestRateIndex
    spread: float
    gearing: float
    calendar: Qcf.BusinessCalendar  # tuve que agregar esta variable

    def get_accrued_rate(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        """
        Calcula la tasa equivalente desde `self.start_date` a `accrual_date`. La tasa equivalente
        se calcula como:

        (P - 1) * 360 / (accrual_date - self.start_date)

        donde P es el producto de los factores de capitalización de todas las tasas overnight
        entre `start_date` y `accrual_date`. Los valores de esas tasas deben estar almacenados
        en la variable `fixings`.

        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        if self.calendar.next_busy_day(accrual_date) > accrual_date:
            raise ValueError("Acrrual date must be a working day.")
        fecha = self.start_date
        if accrual_date <= fecha:
            return 0.0
        factor = 1.0
        while fecha < accrual_date:
            fecha_siguiente = self.calendar.shift(fecha, 1)
            tasa = self.on_index.get_rate()
            try:
                tasa.set_value(fixings[fecha])
            except Exception as e:
                raise ValueError(
                    f'Se produjo el error: {str(e)}. Fecha de cálculo: {fecha.description(False)}')
            factor *= tasa.wf(fecha, fecha_siguiente)
            fecha = fecha_siguiente
        return self.interest_rate.get_rate_from_wf(factor, self.start_date, accrual_date)

    def get_accrued_interest(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        """
        Calcula el interés (plata) devengado desde `self.start_date` a `accrual_date` utilizando la tasa equivalente
        que se calcula con el método anterior.

        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.

        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        valor_tasa = self.get_accrued_rate(accrual_date, fixings)
        self.interest_rate.set_value(valor_tasa)
        return self.notional * (self.interest_rate.wf(self.start_date, accrual_date) - 1.0)

    def amount(self, fixings: Qcf.time_series = None) -> float:
        """
        Calcula el flujo total al vencimiento (amortización más intereses devengados hasta self.end_date).

        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.

        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        if fixings is None:
            return self.notional * self.interest_rate.wf(self.start_date, self.end_date)
        else:
            return self.get_accrued_interest(self.end_date, fixings) + self.amortization

Construcción de un objeto `Qcf.InterestRateIndex`.

In [6]:
codigo = 'SOFR'
tasa_on = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('0d')
tenor = Qcf.Tenor('1d')
fixing_calendar = get_cal(BusCal.NY)     
settlement_calendar = fixing_calendar  
sofr = Qcf.InterestRateIndex(
    codigo,
    tasa_on,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    Qcf.QCUSD()
)

Construcción de una instancia de `OisCashflow`.

In [7]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(1, 10, 2020),
    Qcf.QCDate(1, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1,
    settlement_calendar
)

## Funciones

### `set_expected_rate(...)`

Las funciones que hacen este trabajo en `QC_Finacial_3` no retornan ningún valor, pero cambian el estado del cashflow, al dejar almacenado dentro de él el valor de la tasa flotante (o el valor de los índices ICP). En la clase `OisCashflow` este cambio de estado puede lograrse cambiando el valor de la tasa en el objeto `interest_rate`.

In [8]:
def set_expected_rate(
        val_date: Qcf.QCDate,
        ois_cashflow: OisCashflow,
        fixings: Qcf.time_series,
        zcc: Qcf.ZeroCouponCurve) -> None:
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Ver por ejemplo los casos cuando usamos el objeto Qcf.ForwardRates() en el notebook 9.

    Para un OIS calculando en t0 un cashflow entre (t0 <) t1 < t2
    ICP(t0) * wf(to, t2) = ICP(t2), ICP(t0) * w(t0, t1) = ICP(t1) -> ICP(t2) / ICP(t1) = ICP(t1,t2)
    tasa(t1, t2) = [ICP(t1, t2) - 1] * 360 / (t2-t1)
    El ICP se puede pensar como el account asociado a la tasa interbancaria chilena, que es el índice
    overnight chileno.

    Cuando t1 < t0 < t2 tener un poquito de ojo ... porque el cashflow ya tiene fixings.

    Se distinguen 3 casos (en todos ellos `zcc.wf()` significa `1 / zcc.get_discount_factor_at()`:

    - `val_date <= ois_cashflow.start_date`: se calculan los dos plazos p1, p2 desde `val_date` hasta
    `ois_cashflow.start_date` y `ois_cashflow.end_date` respectivamente. Se define `account(val_date) = 1.0` y luego,
      - `account(ois_cashflow.start_date) = 1.0 * zcc.wf(p1) := wf1`
      - `account(ois_cashflow.end_date) = 1.0 * zcc.wf(p2) := wf2`
      - `tasa_esperada = (wf2 / wf2 - 1.0) * 360.0 / (p2 - p1)`
      - Finalmente, `ois_cashflow.interest_rate.set_value(tasa_esperada)`.

    - `ois_cashflow.start_date < val_date < ois_cashflow.end_date`:
      - se calcula el plazo `p1` desde `ois_cashflow.end_date` hasta `val_date`.
      - Se calcula `wf1 = 1 + ois_cashflow.get_accrued_interest(val_date, fixings) / ois_cashflow.notional`.
      - se calcula el plazo `p2` desde `val_date` hasta `ois_cashflow.end_date`.
      - Se calcula `wf2 = zcc.wf(p2)`.
      - `tasa_esperada = (wf2 / wf2 - 1.0) * 360.0 / (p1 + p1)`.
      - Finalmente, `ois_cashflow.interest_rate.set_value(tasa_esperada)`.

    - `ois_cashflow.end_date <= val_date`: en este caso el flujo debe ser conocido, y por lo tanto:
      - `tasa_esperada = ois_cashflow.accrued_rate(ois_cashflow.end_date)`.
      - Finalmente, `ois_cashflow.interest_rate.set_value(tasa_esperada)`.
    """
    if val_date < ois_cashflow.start_date:
        p1 = val_date.day_diff(ois_cashflow.start_date)
        p2 = val_date.day_diff(ois_cashflow.end_date)
        wf1 = 1 / zcc.get_discount_factor_at(p1)
        wf2 = 1 / zcc.get_discount_factor_at(p2)
        tasa_esperada = (wf2 / wf1 - 1) * 360 / (p2 - p1)
    elif ois_cashflow.start_date <= val_date < ois_cashflow.end_date:
        p1 = ois_cashflow.start_date.day_diff(val_date)
        wf1 = 1 + ois_cashflow.get_accrued_interest(
            val_date, fixings) / ois_cashflow.notional
        p2 = val_date.day_diff(ois_cashflow.end_date)
        wf2 = 1 / zcc.get_discount_factor_at(p2)
        tasa_esperada = (wf1 * wf2 - 1) * 360 / (p1 + p2)
    else:
        tasa_esperada = ois_cashflow.get_accrued_rate(
            ois_cashflow.end_date, fixings)
    ois_cashflow.interest_rate.set_value(tasa_esperada)
    return

### `present_value(...)`

Para que esta función opere de la misma forma que los métodos de la clase `PresentValue` de `QC_Fimancial_3` se requiere tener un método `amount` en `OisCashflow` que no requiera argumentos y retorne el flujo final utilizando el valor de la tasa que se fijó con la función `set_expected_rate`.

En Python no se puede sobrecargar una función o método de una clase (mismo nombre para una función que tiene más de una firma). Sin embargo, se puede lograr un efecto similar transformando en opcional el argumento de la versión ya implementada de `amount`.

In [9]:
def present_value(
        val_date: Qcf.QCDate,
        ois_cashflow: OisCashflow,
        zcc: Qcf.ZeroCouponCurve) -> float:
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Para probar esta función utilizar la data de que está en data/20201012_built_sofr_zero.xlsx.
    Para traer a valor presente debes respetar la convención de las tasas de la curva.

    NOTA: la variable ois_cashflow debe haber 'pasado' por la función set_expected_rate para que
    tenga los intereses de los fixings futuros.
    """
    plazo = val_date.day_diff(ois_cashflow.settlement_date)
    if plazo <= 0:
        return 0.0
    else:
        return ois_cashflow.amount() * zcc.get_discount_factor_at(plazo)

In [10]:
stop

NameError: name 'stop' is not defined

## Tests

In [13]:
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

In [14]:
amount_tol = 10000
rate_tol = .001

In [15]:
exitos = 0

In [36]:
def suma_exito(resultado, check, tipo):
    global exitos
    if resultado is None:
        return
    else:
        print('Suma medio punto por obtener resultado.')
        exitos += .5
        if tipo == 'monto':
            if abs(resultado - check) < amount_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1
        else:
            if abs(resultado - check) < rate_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1

Construcción de una instancia de `OisCashflow`.

In [17]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(1, 10, 2020),
    Qcf.QCDate(1, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1,
    settlement_calendar
)

En la siguiente variable `exitos` se registra cuantos tests se superan con éxito.

### Objeto `fixings`

In [18]:
df_fixings = pd.read_excel('../data/SOFR-10012019-10292020.xls', sheet_name='nice format')
df_fixings.columns = ['fecha', 'nombre', 'valor']
df_fixings['valor'] /= 100

In [19]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


In [20]:
fixings = Qcf.time_series()
for row in df_fixings.itertuples():
    fixings[Qcf.build_qcdate_from_string(row.fecha)] = row.valor

In [21]:
fixings[Qcf.QCDate(13, 1, 2020)]

0.0154

### `OisCashflow.accrued_rate`

In [22]:
accrued_rate = ois.get_accrued_rate(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued rate: {accrued_rate:.8%}')

accrued rate: 1.04766527%


Se verificará el cálculo usando `df_fixings`.

In [23]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


Se agrega la columna `next_date`, servirá para calcular el factor de capitalización de cada tasa.

In [24]:
def next_date(fecha:str, calendario: Qcf.BusinessCalendar) -> str:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    next_fecha = calendario.shift(qfecha, 1)
    return next_fecha.description(False)

In [25]:
df_fixings['next_fecha'] = df_fixings.apply(
    lambda row: next_date(row['fecha'], settlement_calendar),
    axis=1
)

In [26]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha
0,2020-10-28,SOFR,0.0800%,2020-10-29
1,2020-10-27,SOFR,0.0900%,2020-10-28
2,2020-10-26,SOFR,0.0900%,2020-10-27
3,2020-10-23,SOFR,0.0800%,2020-10-26
4,2020-10-22,SOFR,0.0700%,2020-10-23


Se calcula ahora el factor de capitalización.

In [27]:
def factor_cap(fecha: str, next_fecha: str, valor: float) -> float:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    qnext_fecha = Qcf.build_qcdate_from_string(next_fecha)
    int_rate = Qcf.QCInterestRate(valor, Qcf.QCAct360(), Qcf.QCLinearWf())
    return int_rate.wf(qfecha, qnext_fecha)

In [28]:
df_fixings['factor_capitalizacion'] = df_fixings.apply(
    lambda row: factor_cap(row['fecha'], row['next_fecha'], row['valor']),
    axis=1
)

In [29]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha,factor_capitalizacion
0,2020-10-28,SOFR,0.0800%,2020-10-29,1.000002
1,2020-10-27,SOFR,0.0900%,2020-10-28,1.000002
2,2020-10-26,SOFR,0.0900%,2020-10-27,1.000002
3,2020-10-23,SOFR,0.0800%,2020-10-26,1.000007
4,2020-10-22,SOFR,0.0700%,2020-10-23,1.000002


Veamos donde están en `df_fixings` los valores para `start_date` de `ois` y la fecha 15-6-2020 y calculamos el producto de los factores de capitalización desde esa fecha hasta la última fecha de `df_flujos` (que es el primer registro). Luego, el cociente de los dos factores nos dará el factor entre ambas fechas. Finalmente, con ese factor, se calculará `accrued_rate`.

In [30]:
start_date = ois.start_date.description(False)
df_fixings[df_fixings.fecha == start_date]

,fecha,nombre,valor,next_fecha,factor_capitalizacion
269,2019-10-01,SOFR,0.0188,2019-10-02,1.000052


In [31]:
factor_largo = df_fixings.iloc[:270]['factor_capitalizacion'].prod()
print(factor_largo)
qstart_date = Qcf.build_qcdate_from_string('2019-10-01')

1.007853957449575


In [32]:
df_fixings[df_fixings.fecha == '2020-06-15']

,fecha,nombre,valor,next_fecha,factor_capitalizacion
94,2020-06-15,SOFR,0.0009,2020-06-16,1.000002


In [33]:
factor_corto = df_fixings.iloc[:95]['factor_capitalizacion'].prod()
qaccrued_date = Qcf.build_qcdate_from_string('2020-06-15')

In [34]:
dias = qstart_date.day_diff(qaccrued_date)
check_accrued_rate = (factor_largo / factor_corto - 1.0) * 360.0 / dias
print(f'La diferencia es: {accrued_rate - check_accrued_rate:.8%}')

La diferencia es: 0.00000000%


In [37]:
suma_exito(accrued_rate, check_accrued_rate, 'tasa')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `OisCashflow.accrued_interest`

In [38]:
accrued_interest = ois.get_accrued_interest(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued interest: {accrued_interest:,.2f}')

accrued interest: 75,082.68


In [39]:
check_accrued_interest = ois.notional * accrued_rate * dias / 360.0
print(f'La diferencia es: {accrued_interest - check_accrued_interest:,.6f}')

La diferencia es: 0.000000


In [40]:
suma_exito(accrued_interest, check_accrued_interest, 'monto')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `OisCashflow.amount`

In [41]:
amount = ois.amount(fixings)
print(f'amount: {amount:,.2f}')

amount: 1,077,850.90


In [42]:
end_date = ois.end_date.description(False)
print(f'end date: {end_date}')
df_fixings[df_fixings.fecha == end_date]

end date: 2020-10-01


,fecha,nombre,valor,next_fecha,factor_capitalizacion
18,2020-10-01,SOFR,0.0008,2020-10-02,1.000002


In [43]:
factor_corto_2 = df_fixings.iloc[:19]['factor_capitalizacion'].prod()
dias_2 = qstart_date.day_diff(ois.end_date)
accrued_rate_amount = (factor_largo / factor_corto_2- 1.0) * 360.0 / dias_2
check_amount = ois.notional * accrued_rate_amount * dias_2 / 360.0 + ois.amortization
print(f'La diferencia es: {amount - check_amount:.6f}')

La diferencia es: 0.000000


In [44]:
suma_exito(amount, check_amount, 'monto')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `set_expected_rate`

Comenzamos cargando los valores de la curva cero cupón que se utilizará para hacer pruebas de `set_expected_rate` y `present_value`.

In [45]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [46]:
df_curva.head().style.format({'tasa': '{:.4%}', 'df': '{:.6%}'})

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


Con la data se construye un objeto de tipo `Qcf.ZeroCouponCurve`.

In [47]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Comienzan los tests.

In [48]:
print(f'Fecha inicial flujo: {ois.start_date.description(False)}')
print(f'Fecha final flujo: {ois.end_date.description(False)}')

Fecha inicial flujo: 2019-10-01
Fecha final flujo: 2020-10-01


In [49]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.06121277%
Check tasa esperada es: 0.06121277%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [50]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.06924275%
Check tasa esperada es: 0.06924275%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [51]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = ois.start_date.day_diff(val_date)
p2 = val_date.day_diff(ois.end_date)
wf1 = 1 + ois.get_accrued_interest(val_date, fixings) / ois.notional
wf2 = 1 / zcc.get_discount_factor_at(p2)
check = (wf1 * wf2 - 1) * 360 / (p1 + p2)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.22166384%
Check tasa esperada es: 0.22166384%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [52]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(val_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.76574655%
Check tasa esperada es: 0.76574655%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [53]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(ois.end_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.76574655%
Check tasa esperada es: 0.76574655%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `present_value`

In [54]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 9,997,383.0229
Amount: 10,006,223.2979
Df: 99.911652%
VP check es: 9,997,383.0229
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [55]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 10,000,000.0000
Amount: 10,007,039.6797
Df: 99.929653%
VP check es: 10,000,000.0000
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [56]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 10,016,023.1254
Amount: 10,022,535.8238
Df: 99.935019%
VP check es: 10,016,023.1254
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [57]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 0.0000
Amount: 10,077,850.8996
Df: 0.000000%
VP check es: 0.0000
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [58]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 0.0000
Amount: 10,077,850.8996
Df: 0.000000%
VP check es: 0.0000
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [59]:
print(f'Éxitos totales: {exitos}')

Éxitos totales: 19.5
